<a href="https://colab.research.google.com/github/myDSMLProjects/Deep-Learning/blob/master/NLP_Disaster_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rahulvv","key":"9179faa82c7c88b962e8fd802e61a571"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c nlp-getting-started

  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 31.7MB/s]
  0% 0.00/411k [00:00<?, ?B/s]
100% 411k/411k [00:00<00:00, 57.6MB/s]
  0% 0.00/22.2k [00:00<?, ?B/s]
100% 22.2k/22.2k [00:00<00:00, 22.8MB/s]


In [ ]:
#!kaggle datasets download -d watts2/glove6b50dtxt
!kaggle datasets download -d jdpaletto/glove-global-vectors-for-word-representation

100% 1.45G/1.45G [00:27<00:00, 46.8MB/s]
100% 1.45G/1.45G [00:27<00:00, 55.7MB/s]


In [ ]:
import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import string

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.models import Sequential
import os

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from zipfile import ZipFile
file_path = '/content/glove-global-vectors-for-word-representation.zip'
zip_inst = ZipFile(file_path, 'r')
zip_inst.extractall()
zip_inst.close()

In [ ]:
print('Loading word vectors...')
word2vec = {}
with open(os.path.join('/content/glove.twitter.27B.100d.txt'), encoding = "utf-8") as f:
  # is just a space-separated text file in the format:
  # word vec[0] vec[1] vec[2] ...
  for line in f:
    values = line.split() #split at space
    word = values[0]
    vec = np.asarray(values[1:], dtype='float32') #numpy.asarray()function is used when we want to convert input to an array.
    word2vec[word] = vec
print('Found %s word vectors.' % len(word2vec))

Loading word vectors...
Found 1193514 word vectors.


In [ ]:
import pandas as pd
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
print(train_df.shape)
print(test_df.shape)

(7613, 5)
(3263, 4)


In [ ]:
train_sentences = train_df.text.values
train_labels = train_df.target.values
test_sentences = test_df.text.values

In [ ]:
import string
import re

#remove urls
def remove_urls(text):
  url = re.compile(r'https?://\S+|www\.\S+')
  return url.sub(r'',text)
  
#remove html tags
def remove_html(text):
  html=re.compile(r'<.*?>')
  return html.sub(r'',text)

# splitting the text
def split_text(text):
  text = text.split()
  return text

 # making lower case words
def lower(text):
  text = [word.lower() for word in text]
  return str(text)

#remove punct
def remove_punct(text):
  text = ''.join([char for char in text if char not in string.punctuation])
  text = re.sub('[0-9]+', ' ', str(text))
  return text

#remove stopwords
def remove_stopwords(text):
  pattern = re.compile(r'\b('+r'|'.join(stopwords.words('english')) + r')\b\s*')
  text = pattern.sub(' ', text)
  return text

#lemmatization
'''lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
  text = lemmatizer.lemmatize(text)
  return text'''

'lemmatizer = WordNetLemmatizer()\ndef lemmatize_words(text):\n  text = lemmatizer.lemmatize(text)\n  return text'

In [ ]:
def final(text):
  
  t0 = remove_urls(text)
  t1 = remove_html(t0)
  t2 = split_text(t1)
  t3 = lower(t2)
  t4 = remove_punct(t3)
  t5 = remove_stopwords(t4)
  #t6 = lemmatize_words(t5)
  return t5

In [ ]:
training_sentences = []
for i in range(len(train_sentences)):
  data = final(train_sentences[i])
  training_sentences.append(data)

testing_sentences = []
for i in range(len(test_sentences)):
  data = final(test_sentences[i])
  testing_sentences.append(data)

In [ ]:
num_words = 20000
embedding_dim = 100
max_length = 32
trunc_type='post'
padding_type='post'
oov_tok = 'OOV'

tokenizer = Tokenizer(num_words=num_words, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
train_sequences = tokenizer.texts_to_sequences(training_sentences)

word_index = tokenizer.word_index
padded_train = pad_sequences(sequences=train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print('Total unique tokens generated: ',len(word_index))
print('Shape of padded train tensor: ', padded_train.shape)

#tokenizer = Tokenizer(num_words=num_words, oov_token=oov_tok)
#tokenizer.fit_on_texts(testing_sentences)
test_sequences = tokenizer.texts_to_sequences(testing_sentences)
padded_test = pad_sequences(sequences=test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print('Shape of padded test tensor: ', padded_test.shape)

Total unique tokens generated:  17057
Shape of padded train tensor:  (7613, 32)
Shape of padded test tensor:  (3263, 32)


In [ ]:
num_words = min(20000, len(word_index)+1)
embedding_matrix = np.zeros((num_words, embedding_dim))

embeddings = []
for word, i in word_index.items():
  if i<20000:
    embeddings = word2vec.get(word)
    if embeddings is not None:
      embedding_matrix[i] = embeddings

In [ ]:
embeddings.shape

(100,)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

    tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=max_length, weights=[embeddings_matrix], trainable=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')

'''model = tf.keras.Sequential([tf.keras.layers.Embedding(input_dim=num_words,
                                                       output_dim = embedding_dim,
                                                       weights=[embedding_matrix],
                                                       input_length=max_length,
                                                       trainable=False),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(1, activation='sigmoid')])'''

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(lr=0.01),
              metrics=['accuracy'])

#checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 32, 100)           1705800   
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 128)           84480     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 64)            41216     
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2049      
Total params: 1,833,545
Trainable params: 127,745
Non-trainable params: 1,705,800
_________________________________________________________________


In [ ]:
history=model.fit(padded_train,
          train_labels,
          batch_size=128,
          epochs=15,
          validation_split=.4)

Epoch 1/15
36/36 [==============================] - 2s 61ms/step - loss: 0.5048 - accuracy: 0.7578 - val_loss: 0.4633 - val_accuracy: 0.7909
Epoch 2/15
36/36 [==============================] - 1s 20ms/step - loss: 0.4079 - accuracy: 0.8268 - val_loss: 0.4882 - val_accuracy: 0.7771
Epoch 3/15
36/36 [==============================] - 1s 20ms/step - loss: 0.3697 - accuracy: 0.8450 - val_loss: 0.5023 - val_accuracy: 0.7853
Epoch 4/15
36/36 [==============================] - 1s 20ms/step - loss: 0.3051 - accuracy: 0.8758 - val_loss: 0.5476 - val_accuracy: 0.7699
Epoch 5/15
36/36 [==============================] - 1s 19ms/step - loss: 0.2300 - accuracy: 0.9089 - val_loss: 0.7357 - val_accuracy: 0.7748
Epoch 6/15
36/36 [==============================] - 1s 19ms/step - loss: 0.1695 - accuracy: 0.9347 - val_loss: 0.8399 - val_accuracy: 0.7571
Epoch 7/15
36/36 [==============================] - 1s 19ms/step - loss: 0.1227 - accuracy: 0.9575 - val_loss: 0.8567 - val_accuracy: 0.7623
Epoch 8/15
36

In [ ]:
pred = model.predict(padded_test)

In [ ]:
pred = (np.round(pred)).astype(int)
pred = pred.reshape(3263)

In [ ]:
my_submission = pd.DataFrame({'id': test_df.id, 'target': pred})

my_submission.to_csv('submission.csv', index=False)

In [ ]:
my_submission['target'].value_counts()

0    2004
1    1259
Name: target, dtype: int64

In [ ]:
'''from matplotlib import pyplot
pyplot.figure(figsize=(12,7))
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()'''

"from matplotlib import pyplot\npyplot.figure(figsize=(12,7))\npyplot.plot(history.history['loss'])\npyplot.plot(history.history['val_loss'])\npyplot.title('model train vs validation loss')\npyplot.ylabel('loss')\npyplot.xlabel('epoch')\npyplot.legend(['train', 'validation'], loc='upper right')\npyplot.show()"

In [ ]:
!kaggle competitions submit -c nlp-getting-started -f submission.csv -m "This is my first submission"

100% 22.2k/22.2k [00:04<00:00, 5.02kB/s]
403 - Your team has used its submission allowance (5 of 5). This resets at midnight UTC (13 hours from now).
